### Building a RAG System with LangChain and ChromaDB

Introduction

Retrieval-Augmented Generation (RAG) is a powerful technique that combines the capabilities of large language models with external knowledge retrieval. This notebook will walk you through building a complete RAG system using:

- LangChain: A framework for developing applications powered by language models

- ChromaDB: An open-source vector database for storing and retrieving embeddings

- OpenAI: For embeddings and language model (you can substitute with other providers)

In [5]:
import os
from dotenv import load_dotenv
from pathlib import Path
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from typing import List
import warnings
warnings.filterwarnings('ignore')

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')


### 1. sample data

In [6]:
sample_docs = [
    """
    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables
    systems to learn and improve from experience without being explicitly
    programmed. There are three main types of machine learning: supervised
    learning, unsupervised learning, and reinforcement learning. Supervised
    learning uses labeled data to train models, while unsupervised learning
    finds patterns in unlabeled data. Reinforcement learning learns through
    interaction with an environment using rewards and penalties.
    """,

    """
    Deep Learning and Neural Networks

    Deep learning is a subset of machine learning based on artificial neural
    networks. These networks are inspired by the human brain and consist of
    layers of interconnected nodes. Deep learning has revolutionized fields
    like computer vision, natural language processing, and speech recognition.
    Convolutional Neural Networks (CNNs) are particularly effective for image
    processing, while Recurrent Neural Networks (RNNs) and Transformers excel
    at sequential data processing.
    """,

    """
    Natural Language Processing (NLP)

    NLP is a field of AI that focuses on the interaction between computers and
    human language. Key tasks in NLP include text classification, named entity
    recognition, sentiment analysis, machine translation, and question answering.
    Modern NLP heavily relies on transformer architectures like BERT, GPT, and T5.
    These models use attention mechanisms to understand context and relationships
    between words in text.
    """
]


save the sample docs in txt file

In [7]:


path = Path("data")
path.mkdir(parents=True, exist_ok=True)

for i, doc in enumerate(sample_docs):
    (path / f"doc_{i}.txt").write_text(doc, encoding="utf-8")

print("✅ Sample docs created")


✅ Sample docs created


### 2. document load

In [8]:
loader = DirectoryLoader(
    path,
    glob='*.txt',
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)

docs = loader.load()
docs

[Document(metadata={'source': 'data\\doc_0.txt'}, page_content='\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables\n    systems to learn and improve from experience without being explicitly\n    programmed. There are three main types of machine learning: supervised\n    learning, unsupervised learning, and reinforcement learning. Supervised\n    learning uses labeled data to train models, while unsupervised learning\n    finds patterns in unlabeled data. Reinforcement learning learns through\n    interaction with an environment using rewards and penalties.\n    '),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='\n    Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural\n    networks. These networks are inspired by the human brain and consist of\n    layers of interconnected nodes. Deep learning has revolutionized fields\n    like computer vision, na

### 3. Splitting docs

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    separators=['\n\n', '\n', '. ', '']
)

splitted_docs = text_splitter.split_documents(docs)

print(f"total docs after split: {len(splitted_docs)}\n")
splitted_docs
# for i, v in enumerate(splitted_docs):
#     print(f"chunk {i}\n {splitted_docs[i].page_content}\n")

total docs after split: 9



[Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables\n    systems to learn and improve from experience without being explicitly\n    programmed. There are three main types of machine learning: supervised\n    learning, unsupervised learning, and reinforcement learning. Supervised'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='learning uses labeled data to train models, while unsupervised learning\n    finds patterns in unlabeled data. Reinforcement learning learns through\n    interaction with an environment using rewards and penalties.'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural\n    networks. Thes

### 4. Create Embedding and store in Vector store ChromaDB

In [10]:
persist_directory = "./chroma_db"

vector_store = Chroma.from_documents(
    documents=splitted_docs,
    embedding=OpenAIEmbeddings(),
    persist_directory=persist_directory,
    collection_name='rag_collection'
)

print(f"vector store created with '{vector_store._collection.count()}' vectors")
print(f'location: {persist_directory}')

vector store created with '9' vectors
location: ./chroma_db


### 5. Test Similarity Search

In [11]:
query = "what are the types of machine learning"

similar_docs = vector_store.similarity_search(query, k=3)
similar_docs

[Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables\n    systems to learn and improve from experience without being explicitly\n    programmed. There are three main types of machine learning: supervised\n    learning, unsupervised learning, and reinforcement learning. Supervised'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks')]

In [12]:
query = "what is NLP?"
similar_docs = vector_store.similarity_search(query, k=5)
similar_docs

[Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Natural Language Processing (NLP)'),
 Document(metadata={'source': 'data\\doc_2.txt'}, page_content='NLP is a field of AI that focuses on the interaction between computers and\n    human language. Key tasks in NLP include text classification, named entity\n    recognition, sentiment analysis, machine translation, and question answering.'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
 Document(metadata={'source': 'data\\doc_2.txt'}, page_content='Modern NLP heavily relies on transformer architectures like BERT, GPT, and T5.\n    These models use attention mechanisms to understand context and relationships\n    between words in text.'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals')]

In [13]:
query = 'what is deep learning?'
similar_docs = vector_store.similarity_search(query, k=3)
similar_docs

[Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural\n    networks. These networks are inspired by the human brain and consist of\n    layers of interconnected nodes. Deep learning has revolutionized fields'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals')]

#### Advanced Similarity Search with Scores

In [14]:
vector_store.similarity_search_with_score(query, k=3)

[(Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
  0.2128422111272812),
 (Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep learning is a subset of machine learning based on artificial neural\n    networks. These networks are inspired by the human brain and consist of\n    layers of interconnected nodes. Deep learning has revolutionized fields'),
  0.22426888346672058),
 (Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals'),
  0.30851638317108154)]

### Understanding Similarity Scores
The similarity score represents how closely related a document chunk is to your query. 
The scoring depends on the distance metric used:

**ChromaDB default: Uses L2 distance (Euclidean distance)**
- Lower scores = MORE similar (closer in vector space)  
- Score of 0 = identical vectors  
- Typical range: 0 to 2 (but can be higher)  

**Cosine similarity (if configured):**
- Higher scores = MORE similar  
- Range: -1 to 1 (1 being identical)  


### 6. Initialize LLM, RAG Chain, Prompt Template

In [15]:
llm = ChatOpenAI(
    model='gpt-4o',
)

llm.predict("What is LLM?")


"LLM can refer to a few different things depending on the context:\n\n1. **Master of Laws (LL.M.)**: This is an advanced, postgraduate academic degree in law. It is typically pursued by individuals who already hold a first degree in law and wish to specialize further in a particular area or gain international legal qualifications.\n\n2. **Large Language Model**: In the context of artificial intelligence and machine learning, LLM refers to large-scale language models like OpenAI's GPT-3, GPT-4, or similar models developed by other organizations. These models are trained on vast amounts of text data and are capable of understanding and generating human-like text, making them useful for a variety of applications such as chatbots, translations, and more.\n\n3. **Logical Link Management**: In telecommunications, LLM can refer to Logical Link Management, which is part of the control functions of a data link that ensures reliable transmission.\n\nIf you have a specific context in mind, please

### Modern RAG Chain

In [16]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

# create retriver
retriever = vector_store.as_retriever(search_kwarg={"k": 3})

retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000211BE029110>, search_kwargs={})

In [17]:
## Create a prompt template
system_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Context: {context}"""

prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    ('human', "{input}")
])

In [18]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse three sentences maximum and keep the answer concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [19]:
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse three sentences maximum and keep the answer concise.\n\nContext: {context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000211BE753310>, async_client=<openai.resour

This chain:
- Takes retrieved documents
- "Stuffs" them into the prompt's {context} placeholder
- Sends the complete prompt to the LLM
- Returns the LLM's response


In [20]:
rag_chain = create_retrieval_chain(retriever, document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000211BE029110>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't 

In [21]:
query = "what is artificial intelligence?"

result = rag_chain.invoke({"input": query})

In [22]:
print(result['answer'])

Artificial intelligence (AI) is a branch of computer science that involves creating systems capable of performing tasks that typically require human intelligence. These tasks include reasoning, learning, problem-solving, perception, and language understanding. AI encompasses various subfields, including machine learning and natural language processing.


### Alternate method to create the RAG

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

custom_prompt = ChatPromptTemplate.from_template("""
                                                 Use the following context to answer the question. If you do not know the answer based on the context, say you do not know.
                                                 Provide specific details from the context to support your answer.

                                                 Context:
                                                 {context}

                                                 Question:
                                                 {question}

                                                 Answer:""")


def format_doc(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_adv = ({"context": retriever, "question": RunnablePassthrough()}
                 | custom_prompt
                 | llm
                 | StrOutputParser())

rag_chain_adv

{
  context: VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000211BE029110>, search_kwargs={}),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n                                                 Use the following context to answer the question. If you do not know the answer based on the context, say you do not know.\n                                                 Provide specific details from the context to support your answer.\n\n                                                 Context:\n                                                 {context}\n\n                                                 Question:\n                                                 {question}\n

In [27]:
rag_chain_adv.invoke("What is AI?")

'The context does not provide a specific definition of AI. However, it mentions that machine learning is a subset of artificial intelligence that allows systems to learn and improve from experience without being explicitly programmed.'

In [28]:
retriever.get_relevant_documents("what is AI?")

[Document(metadata={'source': 'data\\doc_2.txt'}, page_content='NLP is a field of AI that focuses on the interaction between computers and\n    human language. Key tasks in NLP include text classification, named entity\n    recognition, sentiment analysis, machine translation, and question answering.'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine learning is a subset of artificial intelligence that enables\n    systems to learn and improve from experience without being explicitly\n    programmed. There are three main types of machine learning: supervised\n    learning, unsupervised learning, and reinforcement learning. Supervised'),
 Document(metadata={'source': 'data\\doc_1.txt'}, page_content='Deep Learning and Neural Networks'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals')]

### Add new docs in existing vector store

In [ ]:
# Add new documents to the existing vector store
new_txt = """
Reinforcement Learning in Detail

Reinforcement learning (RL) is a type of machine learning where an agent learns to make
decisions by interacting with an environment. The agent receives rewards or penalties
based on its actions and learns to maximize cumulative reward over time. Key concepts
in RL include: states, actions, rewards, policies, and value functions. Popular RL
algorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and
Actor-Critic methods. RL has been successfully applied to game playing (like AlphaGo),
robotics, and autonomous systems.
"""

new_doc = Document(page_content=new_txt, 
                   metadata={"source": "manual", "topic": "reinforcement learning"}
                   )
new_doc

Document(metadata={'source': 'manual', 'topic': 'reinforcement learning'}, page_content='\nReinforcement Learning in Detail\n\nReinforcement learning (RL) is a type of machine learning where an agent learns to make\ndecisions by interacting with an environment. The agent receives rewards or penalties\nbased on its actions and learns to maximize cumulative reward over time. Key concepts\nin RL include: states, actions, rewards, policies, and value functions. Popular RL\nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and\nActor-Critic methods. RL has been successfully applied to game playing (like AlphaGo),\nrobotics, and autonomous systems.\n')

In [32]:
txt_splitter = RecursiveCharacterTextSplitter(chunk_size=300, 
                                              chunk_overlap=30, 
                                              separators=["\n\n", "\n", " ", "."], 
                                              length_function=len)
new_doc_chunk = txt_splitter.split_documents([new_doc])
len(new_doc_chunk)

3

In [33]:
new_doc_chunk

[Document(metadata={'source': 'manual', 'topic': 'reinforcement learning'}, page_content='Reinforcement Learning in Detail'),
 Document(metadata={'source': 'manual', 'topic': 'reinforcement learning'}, page_content='Reinforcement learning (RL) is a type of machine learning where an agent learns to make\ndecisions by interacting with an environment. The agent receives rewards or penalties\nbased on its actions and learns to maximize cumulative reward over time. Key concepts'),
 Document(metadata={'source': 'manual', 'topic': 'reinforcement learning'}, page_content='in RL include: states, actions, rewards, policies, and value functions. Popular RL\nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and\nActor-Critic methods. RL has been successfully applied to game playing (like AlphaGo),\nrobotics, and autonomous systems.')]

In [34]:
vector_store.add_documents(new_doc_chunk)

['840beff1-8dd1-49bf-bad3-80bf67c5fa5d',
 '8d76a685-a835-4625-b6d6-7aa34d8ae210',
 '00949937-369d-4898-91d9-6da0552c7508']

In [35]:
rag_chain_adv.invoke("what is reinforcement learning?")

'Reinforcement learning (RL) is a type of machine learning where an agent learns to make decisions by interacting with an environment. The agent receives rewards or penalties based on its actions and learns to maximize cumulative reward over time. Key concepts in RL include states, actions, rewards, policies, and value functions. Popular RL algorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and Actor-Critic methods. RL has been successfully applied to game playing (like AlphaGo), robotics, and autonomous systems.'

In [36]:
retriever.get_relevant_documents("what is reinforcement learning?")

[Document(metadata={'topic': 'reinforcement learning', 'source': 'manual'}, page_content='Reinforcement learning (RL) is a type of machine learning where an agent learns to make\ndecisions by interacting with an environment. The agent receives rewards or penalties\nbased on its actions and learns to maximize cumulative reward over time. Key concepts'),
 Document(metadata={'source': 'manual', 'topic': 'reinforcement learning'}, page_content='Reinforcement Learning in Detail'),
 Document(metadata={'topic': 'reinforcement learning', 'source': 'manual'}, page_content='in RL include: states, actions, rewards, policies, and value functions. Popular RL\nalgorithms include Q-learning, Deep Q-Networks (DQN), Policy Gradient methods, and\nActor-Critic methods. RL has been successfully applied to game playing (like AlphaGo),\nrobotics, and autonomous systems.'),
 Document(metadata={'source': 'data\\doc_0.txt'}, page_content='Machine Learning Fundamentals')]

### Conversational Memory

In [39]:
from langchain.chains import create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question which might reference context 
in the chat history, formulate a standalone question which can be understood without the chat history. Do not answer 
the question, just formulate it if needed and otherwise return it as is.
"""

contextualize_q_system_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])
contextualize_q_system_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[l

In [40]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_system_prompt
)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000211BE029110>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIM

In [44]:
qa_system_prompt = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context
to answer the question. If you do not know the answer, just say 'I don't know'. Use three sentences maximum and keep 
the answer concise.

Context: {context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ('system', qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

qa_chain = create_stuff_documents_chain(llm, qa_prompt)
qa_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[l

In [51]:
conversational_rag = create_retrieval_chain(history_aware_retriever, qa_chain)
conversational_rag

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000211BE029110>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typi

In [ ]:
chat_history = []

result1 = conversational_rag.invoke({
    "chat_history": chat_history,
    "input": "what is machine learning?"
})

print("Q: what is machine learning?")
print(f"A: {result1['answer']}")

Q: what is machine learning?
A: Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed.


In [55]:
chat_history.extend([
    HumanMessage(content="What is machine learning?"),
    AIMessage(content=result1['answer'])
])

In [57]:
result2 = conversational_rag.invoke({
    "chat_history": chat_history,
    "input": "What are it's main types?"
})

print("Q: What are it's main types?")
print(f"A: {result2['answer']}")

Q: What are it's main types?
A: The main types of machine learning are supervised learning, unsupervised learning, and reinforcement learning.


In [ ]:
chat_history.extend([
    HumanMessage(content="What are it's main types"),
    AIMessage(content=result2['answer'])
])